Link to proposal: https://docs.google.com/document/d/11em-G0ZfHq5VuA_rN9nDGHgqU8Av-Nze-JRNZ48lhhY/edit?usp=sharing

Link to milestone 1: https://github.com/emma-mens/adaptive-federated-learning/blob/master/notebooks/561%20Milestone%201.ipynb

For milestone 2, the plan is to measure the effect of various networking characteristics on the distributed algorithm.

- To explore the impact of network characteristics on the performance of the distributed algorithm. In particular, I would like to simulate network degradation in some form to emulate low performance in rural parts of developing countries [2G and 3G]. Then I would like to study the effects of each degradation parameter on network algorithm performance. Main characteristics will be: latency, packet loss rate and data rate.

Hypothesis:

1. Decreasing the quality of network characteristics will lead to slower convergence time of the algorithm (or for a fixed time range, worse performance of the algorithm)
2. Networking concepts (such as congestion control) could help alleviate some of the impact on the algorithm from decay in characteristics such as loss rate, bandwidth and latency.

Experimental Setup:

The default values on Amazon Linux for the network parameters were:
1. Bandwidth: 4 Mb/s
2. Loss: 0%
3. Latency: 0.041ms

To simulate network degradation, I impose any network change only on the central aggregator since workers don't communicate among themselves. The assumption is that if there is loss introduced at the ingress and egress of the aggregating server, both ways of the connection between the aggregator and a worker will experience the degradation so applying it only on the aggregating server should be sufficient.

These are the values experimented with:
- Find default values for the network x’tics
- Performance by bandwidth (2G:100kb/s, 200kb/s, 400kb/s; 3G:0.5Mb/s, 2Mb/s, 5Mb/s)
- Performance by loss (5%, 10%, 20%)
- Performance by latency (2G:300ms, 500ms, 1000ms; 3G:100ms, 200ms)

Reference for `tc` can be found [here](https://tldp.org/HOWTO/html_single/Traffic-Control-HOWTO/) and a quicker introduction [here](https://netbeez.net/blog/how-to-use-the-linux-traffic-control/)

Defaults:

```
[ec2-user@ip-172-31-40-120 ~]$ iperf -c localhost -p 51000
------------------------------------------------------------
Client connecting to localhost, TCP port 51000
TCP window size: 2.50 MByte (default)
------------------------------------------------------------
[  3] local 127.0.0.1 port 38562 connected with 127.0.0.1 port 51000
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0-10.3 sec  4.81 MBytes  3.92 Mbits/sec
```
Bandwidth (Mbits/sec): (3.92, 3.98, 3.92, 3.95, 3.95)

```
[ec2-user@ip-172-31-40-120 ~]$ nmap -p 51000 localhost

Starting Nmap 6.40 ( http://nmap.org ) at 2021-03-05 00:16 UTC
Nmap scan report for localhost (127.0.0.1)
Host is up (0.000041s latency).
PORT      STATE SERVICE
51000/tcp open  unknown

Nmap done: 1 IP address (1 host up) scanned in 0.03 seconds
```
Latency(microsec): (41, 40, 42, 42, 48)

```
[ec2-user@ip-172-31-40-120 ~]$ ping -c 100 localhost
...
--- localhost ping statistics ---
100 packets transmitted, 100 received, 0% packet loss, time 101380ms
rtt min/avg/max/mdev = 0.027/0.039/0.044/0.004 ms
```

Loss rate: 0%

Default localhost rules:
```
[ec2-user@ip-172-31-40-120 ~]$ tc qdisc show dev lo
qdisc noqueue 0: root refcnt 2 
```


Changing localhost setting:
```
[ec2-user@ip-172-31-40-120 ~]$ sudo tc qdisc add dev lo root netem delay 300ms
[ec2-user@ip-172-31-40-120 ~]$ ping -c 100 localhost
--- localhost ping statistics ---
100 packets transmitted, 100 received, 0% packet loss, time 99081ms
rtt min/avg/max/mdev = 600.181/600.230/600.320/0.395 ms
#delete
[ec2-user@ip-172-31-40-120 ~]$ sudo tc qdisc del dev lo root
```

```
[ec2-user@ip-172-31-40-120 ~]$ sudo tc qdisc del dev lo root; sudo tc qdisc add dev lo handle 1:0 root dsmark indices 1 default_index 0; sudo tc qdisc add dev lo handle 2:0 parent 1:0 tbf burst 20480 limit 20480 mtu 1514 rate 12500bps
[ec2-user@ip-172-31-40-120 ~]$ sudo tc -s qdisc show dev lo
qdisc dsmark 1: root refcnt 2 indices 0x0001 default_index 0x0000 
 Sent 0 bytes 0 pkt (dropped 0, overlimits 0 requeues 0) 
 backlog 0b 0p requeues 0
qdisc tbf 2: parent 1: rate 100Kbit burst 20Kb lat 0us 
 Sent 0 bytes 0 pkt (dropped 0, overlimits 0 requeues 0) 
 backlog 0b 0p requeues 0
```

```
[ec2-user@ip-172-31-40-120 ~]$ sudo tc qdisc add dev lo root netem loss 5%
[ec2-user@ip-172-31-40-120 ~]$ ping -c 100 localhost
--- localhost ping statistics ---
100 packets transmitted, 87 received, 13% packet loss, time 101354ms
rtt min/avg/max/mdev = 0.031/0.041/0.073/0.009 ms
```
loss_exp, loss_act (5, 13), (10, 22), (20,34): Use half of desired loss in command during experiments.

In [ ]:
sudo tc qdisc del dev lo root; sudo tc qdisc add dev lo handle 1:0 root dsmark indices 1 default_index 0; sudo tc qdisc add dev lo handle 2:0 parent 1:0 tbf burst 20480 limit 20480 mtu 1514 rate 32000bps

In [2]:
import itertools

In [4]:
bandwidth = [100,200,400]
loss = [5,10,20]
latency = [300,500,1000]
all_combinations = list(itertools.product(bandwidth, latency))

Results:

The plan was to fill the 2G interaction table below with the corresponding completion time to get a sense of how degraded communication channel characteristics affected training time. However, I was only able to get the traffic control tool integrated in my experiments by adjusting only one characteristic at a time.

Using gamma=3, tau_max=200, time_budget=60

|Bandwidth(Kb/s)|Loss(%)|Latency|Accuracy|Wall time|
|--|--|--|--|--|
|-|-|300|0.877|297.359|
|-|-|500|0.8755|465.042|
|-|-|1000|0.8755|831.553|
|1000000|-|-|0.8759|97.781|
|1000|-|-|0.8752|254.055|
|100|-|-|0.8758|2022.105|
|-|5|-|0.8757|145.206|
|-|10|-|0.8753|179.623|
|-|10|-|0.8756|221.851|

TODO: Test interactions for 2G

|Bandwidth|Loss|Latency|Accuracy|Wall time|
|--|--|--|--|--|
|100|5|300|||
|100|5|500|||
|100|5|1000|||
|100|10|300|||
|100|10|500|||
|100|10|1000|||
|100|20|300|||
|100|20|500|||
|100|20|1000|||
|200|5|300|||
|200|5|500|||
|200|5|1000|||
|200|10|300|||
|200|10|500|||
|200|10|1000|||
|200|20|300|||
|200|20|500|||
|200|20|1000|||
|400|5|300|||
|400|5|500|||
|400|5|1000|||
|400|10|300|||
|400|10|500|||
|400|10|1000|||
|400|20|300|||
|400|20|500|||
|400|20|1000|||

Discussion:

Quite some time went into trying to get dummynet to work on amazon linux. I eventually found that it was much easier to use `tc` so that is what I moved forward with. `tc` documentation can be found [here](https://tldp.org/HOWTO/html_single/Traffic-Control-HOWTO/) Learning how to use this tool and verify it was also tricky. I got only some basic results with varying bandwidth, loss and latency but not interactions.

The initial results suggest that as expected, under tougher restrictions of bandwidth, loss and latency, it takes a longer time for convergence of the algorithm. Especially, if we use 100Kb/s it takes about 20 times longer compared to if the connection was 1Gb/s.

Next, in addition to implementing compression in messages sent (like reducing the size of the learning model), I will stretch to fill experiment with interaction between bandwidth, loss and latency on training as well as potentially make measurements in a heterogenous setting where not all nodes have the same compute power.

End of Doc

Misc on some commands I was trying

sudo tc qdisc del dev lo root; \
sudo tc qdisc add dev lo handle 1: root dsmark indices 1 default_index 0; \
sudo tc qdisc add dev lo handle 2:0 parent 1: tbf burst 20480 limit 20480 mtu 1514 rate 12500bps; \
sudo tc qdisc add dev lo handle 3:0 parent 2:0 netem loss 2.5%; \
sudo tc qdisc add dev lo handle 4:0 parent 3:0 netem delay 150ms; 

sudo tc qdisc del dev lo root; \
sudo tc qdisc add dev lo handle 1: root dsmark indices 1 default_index 0; \
sudo tc qdisc add dev lo handle 2:0 parent 1: tbf burst 20480 limit 20480 mtu 1514 rate 125000bps; \
sudo tc qdisc add dev lo handle 3:0 parent 2:0 netem loss 2.5%; \
sudo tc qdisc add dev lo handle 4:0 parent 3:0 netem delay 150ms; 

sudo tc qdisc add dev lo root netem delay 300ms

sudo tc qdisc del dev lo root; \
sudo tc qdisc add dev lo handle 1: root dsmark indices 1 default_index 0; \
sudo tc qdisc add dev lo handle 3:0 parent 1: netem delay 250ms; 

sudo tc -s qdisc show dev lo

sudo tc qdisc del dev lo root;
sudo tc qdisc add dev lo root handle 1: prio priomap 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
sudo tc qdisc add dev lo parent 1:2 handle 21: netem loss 2.5%;
sudo tc filter add dev lo parent 1:0 protocol ip u32 match ip sport 51000 0xffff flowid 1:2

sudo tc qdisc del dev lo root; \
sudo tc qdisc add dev lo handle 1: root htb default 11; \
sudo tc class add dev lo parent 1: classid 1:1 htb rate 1000Mbps; \
sudo tc class add dev lo parent 1:1 classid 1:11 htb rate 1Gbit; \
sudo tc filter add dev lo parent 1: protocol ip u32 match ip sport 5001 0xffff flowid 1:11;

sudo tc qdisc del dev lo root; \
sudo tc qdisc add dev lo handle 1: root htb default 11; \
sudo tc class add dev lo parent 1: classid 1:1 htb rate 1000Mbps; \
sudo tc class add dev lo parent 1:1 classid 1:11 htb rate 4Mbit; \
sudo tc qdisc add dev lo parent 1:11 handle 10: netem delay 50ms; \
sudo tc filter add dev lo parent 1: protocol ip u32 match ip sport 51000 0xffff flowid 1:11;

sudo tc qdisc del dev lo root; \
sudo tc qdisc add dev lo root handle 1: prio priomap 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; \
sudo tc qdisc add dev lo parent 1:2 handle 20: tbf burst 20480 limit 20480 mtu 1514 rate 32000bps; \
sudo tc qdisc add dev lo parent 1:2 handle 21: netem delay 300ms; \
sudo tc qdisc add dev lo parent 1:2 handle 22: netem loss 5%; \
sudo tc filter add dev lo parent 1:0 protocol ip u32 match ip sport 51000 0xffff flowid 1:2

sudo tc qdisc del dev lo root; sudo tc qdisc add dev lo handle 1:0 root dsmark indices 1 default_index 0;
    sudo tc qdisc add dev lo handle 2:0 parent 1:0 tbf burst 20480 limit 20480 mtu 1514 rate 32000bps

- Performance by congestion control on worst setting above ([bbr](https://www.techrepublic.com/article/how-to-enable-tcp-bbr-to-improve-network-speed-on-linux/) vs cubic vs reno)